In [1]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeThreeModelsByScores

In [2]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()
URM_stacked = combine_matrices(ICM=ICM, URM=URM)

In [3]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_stacked, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.85)

In [4]:
import numpy as np 
import scipy.sparse as sps

profile_length = np.ediff1d(sps.csr_matrix(URM).indptr)
profile_length, profile_length.shape

(array([168, 609, 140, ..., 260, 182, 329], dtype=int32), (13650,))

In [5]:
block_size = int(len(profile_length)*0.20)
block_size

2730

In [6]:
sorted_users = np.argsort(profile_length)
sorted_users

array([1876, 9021, 5660, ..., 1141, 8526,  317])

In [7]:
len(sorted_users)

13650

In [8]:
cold_start_pos = 0
cold_end_pos = block_size
cold_users_group = sorted_users[cold_start_pos:cold_end_pos]

In [9]:
len(cold_users_group)

2730

In [10]:
warm_users_group = sorted_users[cold_end_pos:]

In [11]:
len(warm_users_group)

10920

In [13]:
### RP3Beta Recommender
RP3Beta_recommender = RP3betaRecommender(URM_train=URM_stacked)
RP3Beta_recommender.fit(topK=181, alpha=0.5709402717259106, beta=0.3566066452521787, implicit=True, normalize_similarity=True)

## IALS Recommender
IALS_recommender = IALSRecommender(URM_train=URM_stacked)
IALS_recommender.fit(epochs=50, num_factors=40, alpha=0.7917156202136513, reg=1.0)

##SLIMELASTICNET Recommender
SLIME_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_stacked)
SLIME_recommender.fit(topK=420, l1_ratio=0.06061216785568925, alpha=0.00554982938879437, workers = 10)

IALSRecommenderLinear: Epoch 1 of 50. Elapsed time 10.20 sec
IALSRecommenderLinear: Epoch 2 of 50. Elapsed time 22.84 sec
IALSRecommenderLinear: Epoch 3 of 50. Elapsed time 31.38 sec
IALSRecommenderLinear: Epoch 4 of 50. Elapsed time 38.74 sec
IALSRecommenderLinear: Epoch 5 of 50. Elapsed time 44.68 sec
IALSRecommenderLinear: Epoch 6 of 50. Elapsed time 50.70 sec
IALSRecommenderLinear: Epoch 7 of 50. Elapsed time 56.76 sec
IALSRecommenderLinear: Epoch 8 of 50. Elapsed time 1.04 min
IALSRecommenderLinear: Epoch 9 of 50. Elapsed time 1.14 min
IALSRecommenderLinear: Epoch 10 of 50. Elapsed time 1.24 min
IALSRecommenderLinear: Epoch 11 of 50. Elapsed time 1.35 min
IALSRecommenderLinear: Epoch 12 of 50. Elapsed time 1.45 min
IALSRecommenderLinear: Epoch 13 of 50. Elapsed time 1.56 min
IALSRecommenderLinear: Epoch 14 of 50. Elapsed time 1.66 min
IALSRecommenderLinear: Epoch 15 of 50. Elapsed time 1.76 min
IALSRecommenderLinear: Epoch 16 of 50. Elapsed time 1.86 min
IALSRecommenderLinear: Epo

100%|████████████████████████████████████▉| 18032/18059 [11:12<00:00, 45.57it/s]

In [14]:
hybrid_recommenders = [RP3Beta_recommender, IALS_recommender, SLIME_recommender]

##SLIME_RP3BETA_IALS
SLIM_RP3_IALS_recommender = MergeThreeModelsByScores(URM_stacked,hybrid_recommenders)
SLIM_RP3_IALS_recommender.fit(alpha=0.0, beta=0.3023745211799622, gamma=1.0)



In [15]:
submission_cold =[]
for user in cold_users_group:
    submission_cold.append((user,RP3Beta_recommender.recommend(user,10)))

In [ ]:
submission_warm =[]
for user in warm_users_group:
    submission_warm.append((user,SLIM_RP3_IALS_recommender.recommend(user,10)))

100%|████████████████████████████████████▉| 18056/18059 [11:24<00:00, 26.37it/s]


In [ ]:
len(submission_cold)

In [ ]:
len(submission_warm)

In [ ]:
submission = submission_cold + submission_warm

In [ ]:
len(submission)

In [ ]:
print(submission[10])

In [ ]:
submission.sort()

In [ ]:
print(submission[0])

In [ ]:
for i in range(0,13650):
    print(submission[i])

In [ ]:
from Utils.utils import write_submission

write_submission(submission, name="coldRP3BETA_warmHybrid")